In [56]:
import requests
import pandas as pd

In [57]:
def scrape(fundCode, categoryCode, limit):

    url = 'https://1web.apse1.dgt.api.fwd.com/api/fund/price/historical'

    payload= {"market":"HK","locale":"zh-CN","offset":0,"limit": limit,"fundCode": fundCode,"categoryCode": categoryCode}

    headers={
        "accept": "application/json, text/plain, */*",
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
        "content-length": "92",
        "origin": "https://www.fwd.com.hk",
        "referer": "https://www.fwd.com.hk/",
        "sec-ch-ua": "\"Chromium\";v=\"116\", \"Not)A;Brand\";v=\"24\", \"Google Chrome\";v=\"116\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "cross-site",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36",
    }

    response = requests.post(url, json=payload, headers=headers)

    return response


In [58]:
# first request, scrape 10 items, just to get the total record count
response = scrape('BGNE', 'VY', 10)

In [60]:
totalRecords = response.json()['result']['totalRecords']
totalRecords

2411

In [61]:
# second scrape, to get all the records
response = scrape('BGNE', 'VY', totalRecords)

In [63]:
fundDescription = response.json()['result']['fundDescription']
prices = response.json()['result']['prices']

In [64]:
def prices2df(prices, fundCode, fundDescription):
    df = pd.DataFrame(prices)
    df['fundCode'] = fundCode
    df['fundDescription'] = fundDescription
    df2 = pd.DataFrame(df.price.to_list())
    df3 = df.join(df2)
    df3.drop(columns='price', inplace=True)
    cols = ['fundCode', 'fundDescription', 'valuationDate', 'bidPrice', 'offerPrice']
    df3 = df3[cols]
    return df3

In [65]:
df = prices2df(prices, fundCode, fundDescription)
df

,fundCode,fundDescription,valuationDate,bidPrice,offerPrice
0,BGNE,貝萊德全球基金–可持續能源基金A2,22/09/2023,15.3800,15.3800
1,BGNE,貝萊德全球基金–可持續能源基金A2,21/09/2023,15.4400,15.4400
2,BGNE,貝萊德全球基金–可持續能源基金A2,20/09/2023,15.8400,15.8400
3,BGNE,貝萊德全球基金–可持續能源基金A2,19/09/2023,15.7200,15.7200
4,BGNE,貝萊德全球基金–可持續能源基金A2,18/09/2023,15.7800,15.7800
...,...,...,...,...,...
2406,BGNE,貝萊德全球基金–可持續能源基金A2,08/10/2013,7.6800,7.6800
2407,BGNE,貝萊德全球基金–可持續能源基金A2,07/10/2013,7.6600,7.6600
2408,BGNE,貝萊德全球基金–可持續能源基金A2,04/10/2013,7.6900,7.6900
2409,BGNE,貝萊德全球基金–可持續能源基金A2,03/10/2013,7.7300,7.7300


In [67]:
df.to_excel('FWD fund prices.xlsx', index=False)